In [1]:
#get packages
from Tkinter import *
import Tkinter as Tk
from pandastable import Table, TableModel
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2TkAgg
from matplotlib.figure import Figure
import copy
from collections import Counter
import pandas as pd
import numpy as np
import networkx as nx
import nltk
from nltk.stem import PorterStemmer
from adjustText import adjust_text

In [2]:
#get data
nltk.download('punkt')

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)

ps = PorterStemmer()
G1 = nx.read_gpickle('network_part1v4.pkl')
G2 = nx.read_gpickle('network_part2v4.pkl')
G3 = nx.read_gpickle('network_part3v4.pkl')
G4 = nx.read_gpickle('network_part4v4.pkl')
G5 = nx.read_gpickle('network_part5v4.pkl')
G = nx.compose(G1, G2)
G = nx.compose(G, G3)
G = nx.compose(G, G4)
G = nx.compose(G, G5)

groups = {}
grouplist = []
IDlist = []
x = []
y = []

for node in G.nodes():
    if type(node) == int:
        groups.setdefault(G.node[node]['group'],[]).extend(list(set(G.node[node]['TA_words'])))
        grouplist.append(G.node[node]['group'])
        IDlist.append(G.node[node]['index'])
        x.append(G.node[node]['viz']['position']['x'])
        y.append(G.node[node]['viz']['position']['y'])
        
nodeframe = pd.DataFrame({'ID': IDlist, 'Group': grouplist, 'x': x, 'y': y})
cluster_results = pd.DataFrame(data = {'ID': ['blank'], 'hits': ['blank'], '# papers': ['blank'], 'keywords': ['blank blank blank blank blank']})

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\eprot\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#define functions
def get_input():
    #print(root.place_slaves())
    query = entry1.get()
    query = [ps.stem(item) for item in nltk.word_tokenize(query.lower())]
    mycount = int(listbox1.get(listbox1.curselection()))

    global cluster_dict_perm
    cluster_dict = {}
    cluster_list = []
    count_list = []
    for i in groups:
        allthere = 1
        count = 9999
        for j in query:
            temp = groups[i].count(j)
            allthere = allthere * temp
            if temp < count:
                count = copy.deepcopy(temp)

        if count >= mycount and allthere > 0:
            cluster_dict[i] = {}
            cluster_dict[i]['nodes'] = []
            cluster_dict[i]['count'] = count
            count_list.append(cluster_dict[i]['count'])
            cluster_list.append(i)

    for node in G.nodes():
        if G.node[node]['group'] in cluster_list:
            temp = copy.deepcopy(G.node[node])
            temp.pop('TA', None)
            temp.pop('TA_words', None)
            temp.pop('viz', None)
            cluster_dict[G.node[node]['group']]['nodes'].append(temp)
            
    cluster_results = pd.DataFrame({'ID': cluster_list, 'hits': count_list})
    cluster_results = cluster_results.set_index('ID')
    cluster_results['# papers'] = 0
    cluster_results['keywords'] = ''

    for i in range(1,max(groups)+1):
        if i in cluster_results.index.values:
            cluster_results.loc[i, '# papers'] = len(cluster_dict[i]['nodes'])
            cluster_results.loc[i, 'keywords'] = ' '.join(t[0] for t in Counter(groups[i]).most_common(5))

    nodehits = copy.deepcopy(nodeframe)
    nodehits['bool'] = nodehits['Group'].isin(list(cluster_results.index))
    nodehits = nodehits[nodehits['bool'] == True]
    nodehits['group_color'] = pd.factorize(nodehits['Group'])[0]
    cluster_results['x'] = nodehits.groupby('Group')['x'].mean()
    cluster_results['y'] = nodehits.groupby('Group')['y'].mean()

    fig, ax = plt.subplots(figsize=(5,5))
    ax.axis("off")
    ax.scatter(np.array(nodeframe[['x' ,'y']])[:,0], np.array(nodeframe[['x' ,'y']])[:,1], c="gray", s=1)
    ax.scatter(np.array(nodehits[['x', 'y']])[:,0], np.array(nodehits[['x', 'y']])[:,1], c=list(nodehits['group_color']), cmap = plt.get_cmap("hsv"), s=1)
    texts = [plt.text(list(cluster_results['x'])[i], list(cluster_results['y'])[i], list(set(list(nodehits['Group'])))[i], color='#ff6a00') for i in range(len(list(cluster_results['x'])))]
    adjust_text(texts)

    cluster_results = cluster_results.drop(['x', 'y'], axis=1)
    cluster_results = cluster_results.reset_index()
    
    canvas = FigureCanvasTkAgg(fig, master=root)
    canvas.draw()
    canvas.get_tk_widget().place(x = 200, y = 25)
    
    plt.close()
    
    if len(cluster_dict) > 0:
        pt = Table(f, dataframe=cluster_results, showtoolbar=False, showstatusbar=False)
        pt.show()
        cluster_dict_perm = copy.deepcopy(cluster_dict)
    
def get_input2():
    ID = entry2.get()
    text.delete(1.0, END)
    if len(cluster_dict_perm) > 0:
        for i in cluster_dict_perm[int(ID)]['nodes']:
            for key,val in i.items():
                text.insert(END, str.upper(str(key)))
                text.insert(END, '\n')
                if isinstance(val, list):
                    for j in val:
                        text.insert(END, str(j))
                        text.insert(END, '\n')
                else:
                    text.insert(END, str(val))
                    text.insert(END, '\n')
            text.insert(END, '\n')
    else:
        text.insert(INSERT, 'Please perform a search first. Then you will be able to access its results.')

In [4]:
#create GUI
root = Tk.Tk()
root.wm_title("Search Scielo Publications")
w, h = root.winfo_screenwidth(), root.winfo_screenheight()
root.geometry("%dx%d+0+0" % (w, h))
#Label 1
label1 = Tk.Label(root,text = 'Query')
label1.place(x = 10, y = 0)
label1.config(justify = CENTER)

entry1 = Tk.Entry(root, width = 30)
entry1.place(x = 10, y = 25)

label3 = Tk.Label(root, text="Hit Threshold")
label3.place(x = 10, y = 50)
label3.config(justify = CENTER)

listbox1 = Tk.Listbox(root, width = 30, height = 5)
listbox1.insert(1, 1)
listbox1.insert(2, 2)
listbox1.insert(3, 3)
listbox1.insert(4, 4)
listbox1.insert(5, 5)
listbox1.place(x = 10, y = 75)

button1 = Tk.Button(root, text = 'Search')
button1.place(x = 10, y = 165) 
button1.config(command = get_input)

label4 = Tk.Label(root, text="Cluster ID to Examine")
label4.place(x = 10, y = 225)
label4.config(justify = CENTER)

entry2 = Tk.Entry(root, width = 30)
entry2.place(x = 10, y = 250)

button1 = Tk.Button(root, text = 'Examine Cluster')
button1.place(x = 10, y = 275) 
button1.config(command = get_input2)

fig, ax = plt.subplots(figsize=(5,5))
ax.axis("off")
ax.scatter(np.array(nodeframe[['x' ,'y']])[:,0], np.array(nodeframe[['x' ,'y']])[:,1], c="gray", s=1)

label2 = Tk.Label(root,text = 'Graph of Clusters Found')
label2.place(x = 375, y = 0)
label2.config(justify = CENTER) 

canvas = FigureCanvasTkAgg(fig, master=root)
canvas.draw()
canvas.get_tk_widget().place(x = 200, y = 25)

plt.close()

label3 = Tk.Label(root,text = 'List of Clusters Found')
label3.place(x = 950, y = 0)
label3.config(justify = CENTER)

f = Frame(root)
f.place(x = 700, y = 25)
pt = Table(f, dataframe=cluster_results, showtoolbar=False, showstatusbar=False)
pt.show()

frame = Frame(root, bd=2, relief=SUNKEN, width=550, height=300)

frame.grid_rowconfigure(0, weight=1)
frame.grid_columnconfigure(0, weight=1)
frame.grid_propagate(False)

xscrollbar = Scrollbar(frame, orient=HORIZONTAL)
xscrollbar.grid(row=1, column=0, sticky=E+W)

yscrollbar = Scrollbar(frame)
yscrollbar.grid(row=0, column=1, sticky=N+S)

text = Text(frame, wrap=WORD, bd=0, xscrollcommand=xscrollbar.set, yscrollcommand=yscrollbar.set)
text.insert(INSERT, "Information on each paper in the selected cluster will go here")
text.grid(row=0, column=0, sticky=N+S+E+W)

xscrollbar.config(command=text.xview)
yscrollbar.config(command=text.yview)

frame.place(x = 700, y = 350)

def _quit():
    root.quit()     # stops mainloop
    root.destroy()  # this is necessary on Windows to prevent
                    # Fatal Python Error: PyEval_RestoreThread: NULL tstate

button = Tk.Button(master=root, text='Quit', command=_quit)
button.place(x = 10, y = 325)

root.mainloop()